# Задание

1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек  
    2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/  
    2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [33]:
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [7]:
data = pd.read_excel('./data/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
max_words = 20000
max_len = 150
num_classes = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [9]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [11]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [12]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [14]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:7]

['приложение', 'очень', 'удобно', 'работать', 'удобный', 'вс', 'спасибо']

In [15]:
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [16]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [17]:
data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [19]:
X_train.shape

(14461, 150)

In [22]:
le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

le.classes_

array([0, 1, 2, 3, 4])

### 2.1 предобученные эмбединги

In [24]:
# http://vectors.nlpl.eu/repository/20/186.zip

emb_idxs = {}
with open('./data/model.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        emb_idxs[word] = coefs

print("Found %s word vectors." % len(emb_idxs))

Found 249947 word vectors.


In [26]:
embedding_dim = 200

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [31]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    weights=[embedding_matrix], 
                    output_dim=embedding_dim, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [34]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 6s 213ms/step - loss: 1.5971 - accuracy: 0.6810 - val_loss: 1.5833 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 5s 202ms/step - loss: 1.5715 - accuracy: 0.7051 - val_loss: 1.5576 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 5s 198ms/step - loss: 1.5465 - accuracy: 0.7051 - val_loss: 1.5327 - val_accuracy: 0.7091
Epoch 4/10
26/26 [==============================] - 5s 197ms/step - loss: 1.5221 - accuracy: 0.7051 - val_loss: 1.5086 - val_accuracy: 0.7091
Epoch 5/10
26/26 [==============================] - 5s 197ms/step - loss: 1.4984 - accuracy: 0.7051 - val_loss: 1.4851 - val_accuracy: 0.7091
Epoch 6/10
26/26 [==============================] - 5s 196ms/step - loss: 1.4754 - accuracy: 0.7051 - val_loss: 1.4622 - val_accuracy: 0.7091
Epoch 7/10
26/26 [==============================] - 5s 197ms/step - loss: 1.4530 - accuracy: 0.7051 - val_loss: 1.4401 - val_accuracy: 0.7091
Epoch 

In [36]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 40ms/step - loss: 1.3788 - accuracy: 0.7073
Test score: 1.3788093328475952
Test accuracy: 0.7073249220848083


### 2.2 эмбединги по умолчанию

In [37]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [38]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 4s 152ms/step - loss: 1.2665 - accuracy: 0.6576 - val_loss: 1.0539 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 4s 141ms/step - loss: 0.9506 - accuracy: 0.7079 - val_loss: 0.8376 - val_accuracy: 0.7367
Epoch 3/10
26/26 [==============================] - 4s 142ms/step - loss: 0.7648 - accuracy: 0.7620 - val_loss: 0.7583 - val_accuracy: 0.7671
Epoch 4/10
26/26 [==============================] - 4s 141ms/step - loss: 0.6815 - accuracy: 0.7812 - val_loss: 0.7241 - val_accuracy: 0.7671
Epoch 5/10
26/26 [==============================] - 4s 140ms/step - loss: 0.6278 - accuracy: 0.7906 - val_loss: 0.7053 - val_accuracy: 0.7699
Epoch 6/10
26/26 [==============================] - 4s 143ms/step - loss: 0.5849 - accuracy: 0.8003 - val_loss: 0.6897 - val_accuracy: 0.7719
Epoch 7/10
26/26 [==============================] - 4s 143ms/step - loss: 0.5417 - accuracy: 0.8144 - val_loss: 0.6733 - val_accuracy: 0.7768
Epoch 

In [40]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 30ms/step - loss: 0.6986 - accuracy: 0.7646
Test score: 0.6986313462257385
Test accuracy: 0.764601469039917


Точность на предобученных эмбедингах ниже. Возможно из-за несоответствия набора токенов (словарей)